# Re Check Arnoldi
I am going to make sure I understand the output from Arnoldi.  I am going to use our stripey PDE matrix with a few extra non-zeros.

In [14]:
using LinearAlgebra, KrylovKit, SparseArrays
n=123
A=spzeros(n,n);
for i in 2:n
    A[i,i]=-2.0; A[i,i-1]=1.0; A[i-1,i]=1.0
end
A[1,n]=A[n,1]=1.0;A[1,1]=-2.0
A[4,5]=A[5,4]=-5.0
v=rand(n)

k=24;
iterator = ArnoldiIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
@time expand!(iterator, factorization)
end


(VInt, H, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

  0.000014 seconds (3 allocations: 1.156 KiB)
  0.000011 seconds (2 allocations: 1.156 KiB)
  0.000015 seconds (2 allocations: 1.234 KiB)
  0.000012 seconds (2 allocations: 1.141 KiB)
  0.000010 seconds (2 allocations: 1.391 KiB)
  0.000009 seconds (1 allocation: 1.062 KiB)
  0.000020 seconds (2 allocations: 1.719 KiB)
  0.000023 seconds (2 allocations: 1.203 KiB)
  0.000020 seconds (1 allocation: 1.062 KiB)
  0.000022 seconds (1 allocation: 1.062 KiB)
  0.000028 seconds (2 allocations: 2.391 KiB)
  0.000037 seconds (1 allocation: 1.062 KiB)
  0.000039 seconds (1 allocation: 1.062 KiB)
  0.000035 seconds (1 allocation: 1.062 KiB)
  0.000045 seconds (1 allocation: 1.062 KiB)
  0.000045 seconds (3 allocations: 3.891 KiB)
  0.000029 seconds (1 allocation: 1.062 KiB)
  0.000038 seconds (1 allocation: 1.062 KiB)
  0.000037 seconds (1 allocation: 1.062 KiB)
  0.000038 seconds (1 allocation: 1.062 KiB)
  0.000054 seconds (1 allocation: 1.062 KiB)
  0.000038 seconds (1 allocation: 1.062 KiB)
 

# Checking the output!

The "Hessenberg Matrix" H and the "Orthonormal Basis" V are stored in some funky format!  

In [4]:
H

4×4 KrylovKit.PackedHessenberg{Float64, Vector{Float64}}:
 -0.538224   1.19174  -6.62448e-17   9.831e-17
  1.19174   -3.05841   1.82896      -9.33769e-17
   ⋅         1.82896  -2.23778       4.25725
   ⋅          ⋅        4.25725      -1.67186

In [5]:
VInt

KrylovKit.OrthonormalBasis{Vector{Float64}}([[0.030043400693052547, 0.035246734226876734, 0.01977148706888059, 0.00999638600485033, 0.0850804516288017, 0.07398928690185987, 0.03447097859614454, 0.11368055442475661, 0.043544824062698384, 0.08494709198594323  …  0.12860098483549626, 0.10650686996226838, 0.0798684202651795, 0.0788972663004718, 0.0029165008806411917, 0.15279991769174936, 0.13988088273301427, 0.04860212823907761, 0.037728342800050635, 0.02964456508846018], [0.01760003241791088, -0.001433147123993222, 0.013712471236066375, -0.35263126791167265, -0.08421429979951826, 0.009562261073456467, 0.11519417447244427, -0.07397586294833985, 0.11325907927283228, 0.027040309415826497  …  0.008514630408678798, 0.04428855486874519, 0.05760876594482876, -0.027309066064669988, 0.19084259003123305, -0.0676004173953913, -0.002578115363383518, 0.0894190054649515, 0.01938040399764111, 0.02050625784219444], [0.0010373839359692708, -0.006675471918598616, -0.198535980187832, 0.027142976211574646, 0

You can convert the H to a normal format using Matrix.

In [8]:
Matrix(H)

4×4 Matrix{Float64}:
 -0.538224   1.19174  -6.62448e-17   9.831e-17
  1.19174   -3.05841   1.82896      -9.33769e-17
  0.0        1.82896  -2.23778       4.25725
  0.0        0.0       4.25725      -1.67186

You can not convert the basis to a matrix using matrix.  You need to use the trick up above.

In [9]:
Matrix(VInt)

LoadError: MethodError: no method matching (Matrix{T} where T)(::KrylovKit.OrthonormalBasis{Vector{Float64}})
[0mClosest candidates are:
[0m  (Matrix{T} where T)([91m::Union{QR, LinearAlgebra.QRCompactWY}[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\qr.jl:434
[0m  (Matrix{T} where T)([91m::QRPivoted[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\qr.jl:441
[0m  (Matrix{T} where T)([91m::SuiteSparse.CHOLMOD.Dense{T}[39m) where T at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\SuiteSparse\src\cholmod.jl:990
[0m  ...

The question last week was: How in the heck are they stored and why are they stored that way?  Lets see if we can work out the how for H using the  documentation
https://jutho.github.io/KrylovKit.jl/latest/man/implementation/

In [10]:
H

4×4 KrylovKit.PackedHessenberg{Float64, Vector{Float64}}:
 -0.538224   1.19174  -6.62448e-17   9.831e-17
  1.19174   -3.05841   1.82896      -9.33769e-17
   ⋅         1.82896  -2.23778       4.25725
   ⋅          ⋅        4.25725      -1.67186

In [11]:
H.data

14-element Vector{Float64}:
 -0.5382242814984691
  1.1917356276955264
  1.1917356276955264
 -3.058407654079403
  1.8289552137201512
 -6.624475273886432e-17
  1.8289552137201521
 -2.2377750217592514
  4.257250487317393
  9.831003199012311e-17
 -9.337691210531407e-17
  4.257250487317392
 -1.6718645035063286
  2.3779721799465205

# Lanczos Version
Lets see if the Lanczos version is lighter weight. 

In [15]:
k=44;
iterator = LanczosIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
@time expand!(iterator, factorization)
end

(VInt, T, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

  0.000011 seconds (4 allocations: 1.203 KiB)
  0.000009 seconds (1 allocation: 1.062 KiB)
  0.000004 seconds (1 allocation: 1.062 KiB)
  0.000023 seconds (4 allocations: 1.297 KiB)
  0.000034 seconds (1 allocation: 1.062 KiB)
  0.000010 seconds (1 allocation: 1.062 KiB)
  0.000013 seconds (1 allocation: 1.062 KiB)
  0.000019 seconds (4 allocations: 1.484 KiB)
  0.000018 seconds (1 allocation: 1.062 KiB)
  0.000010 seconds (1 allocation: 1.062 KiB)
  0.000015 seconds (1 allocation: 1.062 KiB)
  0.000023 seconds (1 allocation: 1.062 KiB)
  0.000020 seconds (1 allocation: 1.062 KiB)
  0.000010 seconds (1 allocation: 1.062 KiB)
  0.000020 seconds (1 allocation: 1.062 KiB)
  0.000015 seconds (4 allocations: 1.859 KiB)
  0.000011 seconds (1 allocation: 1.062 KiB)
  0.000026 seconds (1 allocation: 1.062 KiB)
  0.000011 seconds (1 allocation: 1.062 KiB)
  0.000012 seconds (1 allocation: 1.062 KiB)
  0.000024 seconds (1 allocation: 1.062 KiB)
  0.000021 seconds (1 allocation: 1.062 KiB)
  0.00

In [16]:
T

44×44 SymTridiagonal{Float64, Vector{Float64}}:
 -2.15164   1.45179     ⋅          ⋅        …    ⋅          ⋅          ⋅ 
  1.45179  -1.94997    0.930648    ⋅             ⋅          ⋅          ⋅ 
   ⋅        0.930648  -2.01787    1.07148        ⋅          ⋅          ⋅ 
   ⋅         ⋅         1.07148   -1.96987        ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅         0.975095       ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
   ⋅         ⋅          ⋅          ⋅             ⋅          ⋅   

In [18]:
T[1:67]

67-element SparseVector{Float64, Int64} with 5 stored entries:
  [1 ]  =  -2.15164
  [2 ]  =  1.45179
  [45]  =  1.45179
  [46]  =  -1.94997
  [47]  =  0.930648

In [19]:
fieldnames(SymTridiagonal)

(:dv, :ev)

In [21]:
fieldtypes(SymTridiagonal)

(AbstractVector{T} where T, AbstractVector{T} where T)

Hit tab to see what will auto complete! 

In [24]:
T.ev

44-element Vector{Float64}:
 1.4517863432296139
 0.9306475118023718
 1.0714811429115676
 0.9750946384802397
 1.0029577914602104
 0.9689933621590465
 0.9901759811073895
 1.0062802710718244
 1.012527126290215
 0.9636531785447324
 0.999480845606413
 1.0016190901644226
 1.0144996513303934
 ⋮
 0.9967388989889141
 1.075503716994485
 0.8509002252397687
 1.0535574095586977
 1.0153517122755402
 0.9156763272445044
 1.103845168584392
 0.8701894915447802
 1.0204251482963664
 0.9680444052034848
 1.0089337365741942
 0.9942296364025498